In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [63]:
train = pd.read_csv("train.csv")
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [60]:
lis=train.isnull().sum()
lis

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [102]:
for col in train:
    print(train[col].isnull().sum())
    print(col)

0
Loan_ID
13
Gender
3
Married
15
Dependents
0
Education
32
Self_Employed
0
ApplicantIncome
0
CoapplicantIncome
22
LoanAmount
14
Loan_Amount_Term
50
Credit_History
0
Property_Area
0
Loan_Status


In [103]:
len(train)

614

In [123]:
class ensemble(object):
    def __init__(self,data,col_name,logistic_regression=True,knn=True,gbm=True,random_forest=True):
        
        self.original_data=data
        self.data=data
        self.length=len(data)
        # col_name is the name of the column to be prdicted in the data set
        self.col_name=col_name
        # col is the whole column
        self.col=self.original_data[self.col_name].to_frame()
        self.logistic_regression=logistic_regression
        self.knn=knn
        self.gbm=gbm 
        self.random_forst=random_forest
    
    def addMissingValues(self):
        
        for col in self.data:
            
            if self.data[col].isnull().sum() > 0 :
                
                mean = sum(self.data[col][self.data[col].notnull()])/self.length
                self.data[col].fillna(value=mean, inplace=True)
            
    
    def eda(self):
        # dropping the column to be predicted
        self.data=self.data.drop(self.col_name,axis=1)
        cols=list(self.data)
        # converting categorical variables to numeric variables
        # select only the object type column names from the data frame
        df_cat_cols = list(self.data.select_dtypes(include=['object']).copy())
        for k in df_cat_cols:
            if(self.data[k].nunique()>5):
                self.data=self.data.drop(k,axis=1)
                df_cat_cols.remove(k)
        
        # 1- hot encoding for all the categorical variables with less than 4 types of unique values
        # this also removes the original categorical columns
        self.data= pd.get_dummies(self.data , columns = df_cat_cols)
        
        self.addMissingValues()
       
        
        
        
        pass 
    
    def log_reg_algo(self):
        # converting into series for pandas to understand its the outcome var
        self.col=np.ravel(self.col)
        model=LogisticRegression()
        model.fit(self.data,self.col)
        print(model.score(self.data,self.col))
        pass
    
    def knn_algo(self):
        pass
    
    def gbm_algo(self): 
        pass
    
    def rand_for_algo(self):
        pass
    
    def run(self):
        self.eda()
        if(self.logistic_regression == True):
            log_reg_algo()
        
        if(self.knn==True):
            knn_algo()
            
        if(self.gbm==True):
            gbm_algo()
            
        

In [124]:
x=ensemble(train, 'Loan_Status')
x.eda()

In [127]:
x.data.isnull().sum()

ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Dependents_0               0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

In [128]:
x.log_reg_algo()

0.809446254072


In [173]:
type(x.col)

pandas.core.frame.DataFrame

In [52]:
x=train['ApplicantIncome']
x=x.reshape(-1,1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [179]:
model=LogisticRegression()
model.fit(x.data,x.col)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [57]:
model.score(x,Y)
train.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [64]:
list(train.select_dtypes(include=['object']))


['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [59]:
pd.get_dummies(new,columns=['Gender']).head()

,Loan_ID,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,Gender_Female,Gender_Male
0,LP001002,No,0,Graduate,No,Urban,Y,0,1
1,LP001003,Yes,1,Graduate,No,Rural,N,0,1
2,LP001005,Yes,0,Graduate,Yes,Urban,Y,0,1
3,LP001006,Yes,0,Not Graduate,No,Urban,Y,0,1
4,LP001008,No,0,Graduate,No,Urban,Y,0,1


In [174]:
x.data

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,NaN,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
6,2333,1516.0,95.0,360.0,1.0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
7,3036,2504.0,158.0,360.0,0.0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0
8,4006,1526.0,168.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
9,12841,10968.0,349.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0
